In [102]:
import tensorflow as tf

import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam

import sklearn
from sklearn.preprocessing import StandardScaler

import pandas as pd

In [198]:
exact_data = pd.read_excel("../UNT-Thermography-24/fused_silica_data/fused_silica_exact_train.xlsx")

exact_df = pd.DataFrame(exact_data)

X = exact_df[exact_df.columns[11:]]

Y = exact_df[exact_df.columns[0:11]]

print(f"The original data set has {exact_df.shape[0]} rows, and {exact_df.shape[1]} columns. "
      f"We want to predict {target.shape[1]} output variables from {features.shape[1]} features")

exact_df.head()

The original data set has 10000 rows, and 77 columns. We want to predict 11 output variables from 66 features


,layer 1,layer 2,layer 3,layer 4,layer 5,layer 6,layer 7,layer 8,layer 9,layer 10,...,0.000008,0.000008,0.000008,0.000008,0.000008,0.000008,0.000008,0.000008,0.000008,0.000008
0,312.355087,301.649641,321.270850,324.903956,329.245296,337.694194,333.776774,333.750946,331.605971,333.828275,...,2.378753e-12,2.416740e-12,2.452565e-12,2.482976e-12,2.505710e-12,2.514345e-12,2.497372e-12,2.443515e-12,2.360022e-12,2.283099e-12
1,308.045992,314.390077,319.612686,324.021871,328.286597,324.268581,318.045216,310.205743,316.395558,315.206667,...,4.690686e-12,4.758857e-12,4.814785e-12,4.855500e-12,4.879557e-12,4.876002e-12,4.823191e-12,4.700047e-12,4.521287e-12,4.356649e-12
2,275.470643,334.023614,330.292063,333.180051,317.824092,306.451798,309.091647,312.060959,320.208051,336.027107,...,2.891349e-12,2.935824e-12,2.976514e-12,3.009965e-12,3.033863e-12,3.040662e-12,3.016554e-12,2.948046e-12,2.844024e-12,2.748188e-12
3,319.154096,314.324236,322.541381,324.154848,333.263850,344.199198,338.490892,334.185374,327.065180,303.761845,...,1.335647e-12,1.361978e-12,1.387258e-12,1.409588e-12,1.427626e-12,1.437648e-12,1.432962e-12,1.406923e-12,1.363502e-12,1.323517e-12
4,368.613557,330.588867,323.002818,322.824169,336.505619,328.080745,325.609686,335.941694,328.415271,330.880143,...,2.861867e-12,2.906931e-12,2.947792e-12,2.981218e-12,3.005087e-12,3.012002e-12,2.988298e-12,2.920602e-12,2.817710e-12,2.722916e-12


In [207]:
#Normalize features
scaler = sklearn.preprocessing.StandardScaler()
X_scaled = scaler.fit_transform(X)

X_scaled_df = pd.DataFrame(X_scaled, columns=X.columns)

In [296]:
#Goal: Using keras to implement a feed-forward neural network to predict temperatures of each layer given wavelength spectrum
X_train, X_test, Y_train, Y_test = train_test_split(X_scaled_df, Y, train_size = .8, random_state = 1337)

X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, train_size=0.8, random_state = 1337)

print(
    f"Using {len(Y_train)} samples for training, "
    f"and {len(Y_test)} for testing "
    f"from {len(X_test) + len(X_train)} total samples."
)

Using 6400 samples for training, and 2000 for testing from 8400 total samples.


In [300]:
# Define the input shape
input_layer = keras.Input(shape=(X_train.shape[1],))

#Define the model (add more layers? increase nodes?)
exact_model = keras.Sequential([input_layer, #input_layer --> informs input shape
    layers.Dense(256, activation="relu"), # first hidden layer
    layers.Dense(128, activation="relu"), # second hidden layer
    # layers.Dense(20, activation="relu"), # third hidden layer
    # layers.Dense(20, activation="relu"), # fourth hidden layer
    layers.Dense(11)  # Output layer with 11 neurons for each temperature target
])

# Compile the model
# Instantiate optimizer with adjustable learning rate
# observed: sweet spot .02 > x > .001
Adam1 = Adam(learning_rate = 0.009)

# Leverage both MSE and MAE (experimental)
# test different optimizers: SGD (*stochastic gradient descent: better for large data sets)
exact_model.compile(loss="mse", optimizer=Adam1, metrics=["mae"])  # Loss: MSE, Optimizer: Adam w/ MAE

# Train the model
# Thoughts: Overfit then implement regularization
# play around with batch size: batch size for larger datasets more likely to improve its performance
# Dima went up to 60 epochs for GaN_exact: later 600 -> imrpovement plateaus around 120 epochs
# plot loss over epochs
history = exact_model.fit(X_train, Y_train, epochs=15, validation_data = [X_val, Y_val])

#View Training Error
print(f"train_MSE: {history.history["loss"]}")
print(f"train_MAE: {history.history.get("mae", None)}")

# Evaluate the model on the validation set (separate call)
validation_loss, validation_accuracy = exact_model.evaluate(X_val, Y_val)

print(f"val_MSE: {validation_loss}, val_MAE: {validation_accuracy}")

Epoch 1/15
200/200 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - loss: 32274.5566 - mae: 141.7397 - val_loss: 172.9228 - val_mae: 9.5949
Epoch 2/15
200/200 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 186.8254 - mae: 10.0881 - val_loss: 157.2077 - val_mae: 9.0459
Epoch 3/15
200/200 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 210.9210 - mae: 10.4375 - val_loss: 206.3429 - val_mae: 10.8479
Epoch 4/15
200/200 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 163.7638 - mae: 9.2648 - val_loss: 226.1487 - val_mae: 11.6138
Epoch 5/15
200/200 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 175.4882 - mae: 9.5281 - val_loss: 155.7673 - val_mae: 8.5640
Epoch 6/15
200/200 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 232.9620 - mae: 10.2287 - val_loss: 158.7628 - val_mae: 9.3306
Epoch 7/15
200/200 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 141.8321 - mae: 8.1919 - val_loss: 185.9385 - val_mae: 10.2223
Epoch 8/15
200/200 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 137.3368 - mae: 7.9178 - val_loss: 131.7912 - val_mae: 7.6719
Epoch 9/15
200

In [298]:
# Evaluate the model on unseen data
loss, mae = exact_model.evaluate(X_test, Y_test)
print("test_MSE:", loss)
print("test_MAE:", mae)

63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 766us/step - loss: 122.8408 - mae: 7.3795
test_MSE: 122.12621307373047
test_MAE: 7.383547306060791


In [301]:
#TODO:
#plot predictions against actual (scatter plot to see if similar)
#m nodes by n layers: want m*n to be less than number of data points
#per layer residual
#avg (above) to get overall residual
#plot error over epochs
#increase layers: implement loop
#change activation function to hyperbolic tangent: (theoretical difference: ReLu 0 and 1, tangent maintains positive and negative (if normalized data is positive and negative, it is possible tanegent would work better (consider what might happen if all data is positive)))
# 20 layers 250 nodes, batch size 15, epochs 220
#learning rate vs momenta/momento?? may reduce epochs